In [ ]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import re

In [ ]:
banned_content = [
    'บริการส่ง', 'สั่งผ่าน', 'ผ่านแอพ', 'ems', 'ลงทะเบียน', 'แอดไลน์', 'บริการจัดส่ง', 'ผ่านช่องทาง', 'j&t', 'แทรค', 'ปลายทาง', 'สั่งซื้อ', 'ไปรษณีย์', 'นัดรับ', # delivery provided
    'เงื่อนไขเป็นไปตาม', 'สั่งเลย', #promotion
    'แจก', 'การ์ด', 'nct', 'เป๊กผลิตโชค', 'pp', 'พีพี', # fandom  
    'taxi', 'car', 'โดยสาร', 'ค่ารถ', 'แท็กซี่', 'แท็กซี', 'just', 'bike', 'premium', 'คัน', 'bolt', 'โพสต์นี้เพื่อพี่', # transport sector
    'mart', 'express',
    'มือสอง',
    'กัญชา', 'พันลำ', 'พอต', # og
    'นัดเย็ด', # xxx
    'อ่านต่อ' #news
    ]

In [ ]:
!pip install advertools

In [ ]:
import advertools as adv
from copy import copy

In [ ]:
from datetime import datetime, timedelta

In [ ]:
foodpanda_df = pd.DataFrame(columns = ['datetime', 'id', 'text', 'username', 'url'])

In [ ]:
keyword = 'food panda'
start_date = '2021-09-01'
finish_date = '2022-09-30'
offcial_account = 'shopeeth'
max_tweet = 6000

In [ ]:
start_dt = datetime.strptime(start_date, '%Y-%m-%d')
finish_dt = datetime.strptime(finish_date, '%Y-%m-%d')
delta_day = finish_dt - start_dt
start_query_dt = start_dt + timedelta(delta_day.days)
start_query_dt -= timedelta(days=1)
start_query_date = start_query_dt.strftime('%Y-%m-%d')
finish_query_date = finish_date
start_query_date_check = start_query_date

while start_query_date_check != start_date:
    if foodpanda_df.shape[0] == max_tweet:
        break
    start = 0
    query = keyword + ' ' + 'since:' + start_query_date + ' until:' + finish_query_date  + ' lang:th'
    print(start_query_date_check)
    for i, tweet in enumerate(sntwitter.TwitterSearchScraper(query).get_items()):
        # condition1 : if the number of tweet_list exceed max, then stop the loop
        if foodpanda_df.shape[0] == max_tweet: 
            break

        # condition2 : in case keyword's English, the overall content'll be from foriegn provider (e.g. Malaysia, Singapore)
        # so if any content does not contain any letter in thai, we'll skip to the next tweet
        # if tweet.lang != 'th':
        #     continue
        
        pattern0 = 'แกรบ'
        result_thai = re.findall(pattern0, tweet.content.strip().lower())
        if len(result_thai) > 0: 
            continue

        # condition3 : 
        if tweet.user.username.lower() == offcial_account:
            continue

        # condition4 : filter out the tweet that has the number of hashtag exceed 4 (expect it as a delivery information)
        pattern = '#'
        result_hashtag = re.findall(pattern, tweet.content.strip().lower())
        if len(result_hashtag) >= 3: 
            continue

        # condition5 :
        emoji_summary = adv.extract_emoji(tweet.content.strip().lower())
        num_emoji = sum(emoji_summary['emoji_counts'])
        if num_emoji >= 3:
            continue

        # conditon6 : filter out the tweet that contains any word in banned_content
        pattern2 = '|'.join(banned_content)
        result_banned = re.findall(pattern2, tweet.content.strip().lower())
        if len(result_banned) > 0: 
            continue

        # condition7 : filter out the tweet that has user mention
        pattern3 = '@'
        result_tagged = re.findall(pattern3, tweet.content.strip().lower())
        if len(result_tagged) > 0: 
            continue
   
        foodpanda_df = foodpanda_df.append({'datetime':tweet.date, 'id':tweet.id, 'text':tweet.content.strip(), 'username':tweet.user.username, 'url':tweet.url}, ignore_index=True)

        if foodpanda_df.shape[0] % 20 == 0 and foodpanda_df.shape[0] / 20 >= 1:
            break
        
        # display 
        print("(now): ", foodpanda_df.shape[0])
        print(tweet.content.strip())

    start_query_date_check = copy(start_query_date)
    start_query_dt = datetime.strptime(start_query_date, '%Y-%m-%d')
    start_query_dt = start_query_dt - timedelta(days=1)
    finish_query_dt = datetime.strptime(finish_query_date, '%Y-%m-%d')
    finish_query_dt = finish_query_dt - timedelta(days=1)
    start_query_date = start_query_dt.strftime('%Y-%m-%d')
    finish_query_date = finish_query_dt.strftime('%Y-%m-%d')
    print(start_query_date_check)
    print('-----')

In [ ]:
foodpanda_df.shape[0]

1320

In [ ]:
foodpanda_df.to_csv('foodpanda_dataset.csv', index=False)

In [ ]:
robinhood_df.to_csv('robinhood_dataset.csv', index=False)

In [ ]:
robinhood_df.shape[0]

1680

In [ ]:
shopeefood_df.shape[0]

1786

In [ ]:
shopeefood_df.iloc[-5]

datetime                            2021-10-15 05:57:37+00:00
id                                        1448890786263625732
text        เพิ่งรู้ว่า Shopee สั่งอาหารได้ ส่วนลดเยอะมากก...
username                                             wallika_
url         https://twitter.com/wallika_/status/1448890786...
Name: 802, dtype: object

In [ ]:
shopeefood_df.to_csv('shopeefood_dataset.csv', index=False)